在这个小例子里面 我们会学习到：

1）用name scope 去划分 graph
2）写summary 为了可视化
3）用sess 运行

In [1]:
import tensorflow as tf
import numpy as np

# Building the Graph

We need to build 5 sections (name_scopes) at highest level of the graph, under each section, we may insert sub-level section (name_scopes) as needed
1. Variable: to store two variables (non for training): a. count of training times; b. output
2. Transformation: operations of input tensors (e.g. In the later Chapters, this can be Neural Network)
3. Update: update output/count of training times after each training loop
4. summary: create tensor board summary, used for tensor board scalars
5. Global_operations: Initialze, Merge Summary for each training loop to one summary

In [2]:
#this name scope contains variables not for training
with tf.name_scope("variables"):
    #variable indicating how many times the graph (training loop) run
    global_step=tf.Variable(0,dtype=tf.int32,trainable=False,name="global_step")
    
    #Variable of sum of outputs
    total_outputs=tf.Variable(0.0,dtype=tf.float32,trainable=False,name="total_output")

#this name scope contains variables for transformation    
with tf.name_scope("transformation"):
    
    #sub-level name scope1: define input
    with tf.name_scope("input"):
        a=tf.placeholder(tf.float32, shape=(None), name='input_placeholder_a') #shape=(None) means any shape is fine
        
    #sub-level name scope2: define middle operations variables
    with tf.name_scope("middle"):
        b=tf.reduce_prod(a,name='production_b')
        c=tf.reduce_sum(a,name='sum_c')
        
    #sub-level name scope3: define output variable for each transformation
    with tf.name_scope("output"):
        output=tf.add(b,c, name='output')
    

#this name scope update variables:
with tf.name_scope('update'):
    #add output to total_output
    total_outputs_copy=total_outputs.assign_add (output) #add each training loop (transformation)'s output to total_outputs
    global_step_copy=global_step.assign_add(1) #add 1 after each loop to global_step
    
    #please note: assign_add() here will directly change "total_outputs" and "global steps" (referred as xxx and xxx in below), 
    # and return a copy of updated xxx and xxx
    # I used a new variable (total_outputs_copy and global_step_copy) to store the copy of updated value because later on I may change the data type for calculate purpose (e.g. calculate average below)
    # therefore I don't want to directly change on the original variable xxx and xxx, but to change datatype on that copy
    
    ##This treatment above is the best practice in industry##
    
#This name scope create tensor board summaries
with tf.name_scope('summaries'):
    average=tf.div(total_outputs_copy, tf.cast(global_step_copy,tf.float32), name='average') 
    #tf.cast() here: convert dtype of global_step_copy to float32, 
    # reason: tf.div() can only accept input arguments to be the same dtype, but total_outputs_copy was float32 while global_step_copy was int32
    
    
    tf.summary.scalar('output_summary', output)
    tf.summary.scalar('total_summary', total_outputs_copy)
    tf.summary.scalar('average_summary', average)
    
#This name scope initialize variable (globally) and merge summary under each training loop(transformation)
with tf.name_scope('global_operations'):
    #Initialize
    init=tf.global_variables_initializer() #global_variables_initializer can initialize all variable at once, comparing to variables_initializer which can only initialize the specfic variable 
    #Merge all summaries generated under each training loop to one summary
    merged_summaries=tf.summary.merge_all()

In [3]:
#Start a session
sess=tf.Session()
#Open a summary writer
summary_writer=tf.summary.FileWriter('./TFExample1',sess.graph)
#Initialize: run the "init" variable which is previously defined
sess.run(init)

In [4]:
def run_graph(input_tensor):
    feed_dict={a: input_tensor}
    
    #run the following three variables given feed_dict
    outpt,step,summary = sess.run([output,global_step_copy,merged_summaries], feed_dict=feed_dict)
    
    print("step: ", step)
    print("output: ", outpt)
    
    #add something into summary_writer
    summary_writer.add_summary(summary,step)
    summary_writer.add_graph(sess.graph) #note: because in code above we opened a summary writer "tf.summary.FileWriter(xxx, sess.graph)"
                                        # before run session, so we need to update the graph after run session using add_graph(sess.graph)

In [5]:
run_graph([2,8]) #first training loop (transformation): output=b+c=(2x8)+(2+8)=26
run_graph([3,1,1]) #second training loop (transformation): output=b+c=(3x1x1)+(3+1+1)=8

step:  1
output:  26.0
step:  2
output:  8.0


In [6]:
#call this method to make sure that all pending events have been written to disk
summary_writer.flush()

In [7]:
#close
summary_writer.close()
sess.close()

To see Graph and Summary on Tensor Board:
- anaconda prompt: navigate to Example1 folder, run
> tensorboard --logdir=./